In [4]:
from balpy_v2.lib import Chain

from fees_reporting.fees_report_v3 import analyze_pool
from fees_reporting.fees_report_v2 import generate_cycles_until_now

import logging
logging.basicConfig(level=logging.INFO)

import json
import pickle

gauges = json.load(open('gauges.json'))
chain_pools = {
    Chain.mainnet: [gauge["pool"]["id"] for gauge in gauges if gauge['network'] == Chain.mainnet.value],
    Chain.gnosis: [gauge["pool"]["id"] for gauge in gauges if gauge['network'] == Chain.gnosis.value],
    Chain.optimism: [gauge["pool"]["id"] for gauge in gauges if gauge['network'] == Chain.optimism.value]
    # arbitrum: [gauge["pool"]["id"] for gauge in gauges if gauge['network'] == Chain.arbitrum],
    # polygon: [gauge["pool"]["id"] for gauge in gauges if gauge['network'] == Chain.polygon]
}


cycles = generate_cycles_until_now()

# Load previous results if available
try:
    with open('results3.pickle', 'rb') as handle:
        results = pickle.load(handle)
except FileNotFoundError:
    results = []

# Loop over all chains and all pools in each chain
for chain, pool_ids in chain_pools.items():
    logging.info(f"Analyzing pools in chain {chain}...")
    for idx, pool_id in enumerate(pool_ids[:1]):
        logging.info(f"Analyzing #{idx}, pool {pool_id}...")
        try:
            result = await analyze_pool([(pool_id, chain)], cycles[:1])
            results.append(result)
        except Exception as e:
            logging.error(f"Failed to analyze pool {pool_id}: {e}")
            results.append(None)
            raise e

            # continue  # Skip to next pool_id if an error occurred

        # Save results after each pool to not lose any data in case of interruption
        with open('results.pickle', 'wb') as handle:
            pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)


INFO:root:Analyzing pools in chain Chain.mainnet...
INFO:root:Analyzing #0, pool 0x5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014...
INFO:root:Starting pool analysis...
INFO:root:Fetching swaps and joins data...
INFO:root:SWAPS_QUERY Fetching data for 1658718000 - 1657508400 with skip 0, page size 1000, pages per group 1, max pages 1000
DEBUG:root:Executing query: query MySwaps (
DEBUG:root:URL: https://api.thegraph.com/subgraphs/name/bleu-studio/balancer-mainnet-v2
DEBUG:root:Variables: {'after': 1657508400, 'before': 1658718000, 'skip': 0, 'poolId': '0x5c6ee304399dbdb9c8ef030ab642b10820db8f56000200000000000000000014'}
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/joseribeiro/Library/Caches/pypoetry/virtualenvs/balpy-v2-QTj7Rz1O-py3.11/lib/python3.11/site-packages/certifi/cacert.pem'
INFO:root:JOINS_QUERY Fetching data for 1658718000 - 1657508400 with skip 0, page size 1000, pages per gr

In [7]:
results[0]

(                                            id_tokenIn  \
 0    0x700ab55e83ac3c9fde118f94d4535ef841b02ed309c7...   
 1    0xed9f9d61a3f49eaf0f9b3b13733783b5fa2671e86f8b...   
 2    0xad2fd3cb2dac77aaeb376c7f27b6407f91cef19eb985...   
 3    0x5781500565a93bd27b442b32d9e0e2ef1d95ff479d57...   
 4    0x2ef8e4067ba9085eb11e62a5d29bd17bda2b284ed6e0...   
 ..                                                 ...   
 858  0x66d1ea40e108f543ebc57f63a8275b54e9874bbc908f...   
 859  0x99a8bacff03494cf8026a516f1b3de2bb037a7a82e70...   
 860  0x05e9ea29c75de866c34786e41dee24735195583621d1...   
 861  0xf6bd783814123a843ab262a5777d9dcc9ad47710d986...   
 862  0x92c63ce8ea1a388fc283de2fdba5a9faa41b80c50da3...   
 
                         valueUSD_tokenIn                  swapFeesUSD_tokenIn  \
 0    20979.26525335582622337519445031656  209.7926525335582622337519445031656   
 1    4748.833989311984891141511099944807  47.48833989311984891141511099944807   
 2     7578.77884934979240316835098473942   

In [2]:

from operator import itemgetter 
import pandas as pd

# swaps_df, joins_df = await generate_reports(pool_ids, cycles=cycles)
import pickle

with open('results.pickle', 'rb') as handle:
    results = pickle.load(handle)
    

print(len(results))
# print(len(pool_ids))
filtered = list(filter(lambda x: x is not None and not isinstance(x, pd.DataFrame), results))
print(len(filtered))

3
1


In [3]:
dfzao = pd.concat(map(itemgetter(-1), filtered)).sort_index()
dfzao.head()

swapFeeUSD  \
cycle poolId                                             token                                                    
1     0x5c6ee304399dbdb9c8ef030ab642b10820db8f5600020... 0xba100000625a3754423978a60c9317c58a424e3d         0.0   
                                                         0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2         0.0   

                                                                                                     swapFeeTokenAmount  \
cycle poolId                                             token                                                            
1     0x5c6ee304399dbdb9c8ef030ab642b10820db8f5600020... 0xba100000625a3754423978a60c9317c58a424e3d          316.764798   
                                                         0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2            0.841328   

                                                                                                     joinExitFeeUSD  \
cycle poolId                                             token                                                        
1     0x5c6ee304399dbdb9c8ef030ab642b10820db8f5600020... 0xba100000625a3754423978a60c9317c58a424e3d             0.0   
                                                         0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2             0.0   

                                                                                                     joinExitFeeTokenAmount  \
cycle poolId                                             token                                                                
1     0x5c6ee304399dbdb9c8ef030ab642b10820db8f5600020... 0xba100000625a3754423978a60c9317c58a424e3d             6710.499592   
                                                         0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2                0.000000   

                                                                                                     totalUSD  \
cycle poolId                                             token                                                  
1     0x5c6ee304399dbdb9c8ef030ab642b10820db8f5600020... 0xba100000625a3754423978a60c9317c58a424e3d       0.0   
                                                         0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2       0.0   

                                                                                                      totalToken  
cycle poolId                                             token                                                    
1     0x5c6ee304399dbdb9c8ef030ab642b10820db8f5600020... 0xba100000625a3754423978a60c9317c58a424e3d  7027.264391  
                                                         0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2     0.841328